<a href="https://colab.research.google.com/github/lilfetz22/gym-anytrading/blob/master/gym_anytrading/datasets/prepping_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import pandas as pd
from collections import deque
import mplfinance
from renkodf import Renko
from scipy.signal import lfilter
# import MetaTrader5 as mt5

In [ ]:
filepath = "C:/Users/WilliamFetzner/Documents/Trading/"
anytrading_filepath = "/gym-anytrading/gym_anytrading/datasets/data/"
brick_size = 0.00081
brick_size_str = str(int(brick_size*100000))

In [ ]:
# # collect the actual data from csv file
# # filename = "C:/Users/WilliamFetzner/Documents/Trading/DAT_XLSX_EURUSD_M1_2023.xlsx"

# # exported from tickstory

# yr_tickstory_csvs = []
# for year in range(17, 22):
#     yr_tickstory_csvs.append(f"EURUSD_11_15_{year}_to_11_15_{year+1}.csv")
# tickstory_csvs = ["EURUSD_11_15_22_to_11_14_23.csv", "3mo_EURUSD.csv", "EURUSD_2_13_to_3_6.csv",
#                     "EURUSD_3_6_to_3_8_24.csv", "EURUSD_3_8_to_3_12_24.csv", "EURUSD_3_12_to_3_22_24.csv",
#                     "EURUSD_3_23_to_3_30_24.csv"]
# tickstory_csvs.extend(yr_tickstory_csvs)
# tickstory_csvs


['EURUSD_11_15_22_to_11_14_23.csv',
 '3mo_EURUSD.csv',
 'EURUSD_2_13_to_3_6.csv',
 'EURUSD_3_6_to_3_8_24.csv',
 'EURUSD_3_8_to_3_12_24.csv',
 'EURUSD_3_12_to_3_22_24.csv',
 'EURUSD_3_23_to_3_30_24.csv',
 'EURUSD_11_15_17_to_11_15_18.csv',
 'EURUSD_11_15_18_to_11_15_19.csv',
 'EURUSD_11_15_19_to_11_15_20.csv',
 'EURUSD_11_15_20_to_11_15_21.csv',
 'EURUSD_11_15_21_to_11_15_22.csv']

In [ ]:
csvs = [ 'EURUSD_11_15_17_to_11_15_18.csv',
 'EURUSD_11_15_18_to_11_15_19.csv',
 'EURUSD_11_15_19_to_11_15_20.csv',
 'EURUSD_11_15_20_to_11_15_21.csv',
 'EURUSD_11_15_21_to_11_15_22.csv',
 'EURUSD_11_15_22_to_11_14_23.csv',
 '3mo_EURUSD.csv',
 'EURUSD_2_13_to_3_6.csv',
 'EURUSD_3_6_to_3_8_24.csv',
 'EURUSD_3_8_to_3_12_24.csv',
 'EURUSD_3_12_to_3_22_24.csv',
 'EURUSD_3_23_to_3_30_24.csv',
]

In [ ]:
# # df = pd.DataFrame()
# for df_name in last_to_add:
#     # check if filepath + "EURUSD_11_15_17_to_3_30_24.csv" exists
#     if not os.path.exists(filepath + "EURUSD_full_tickstory_data.csv"):
#         df = pd.DataFrame()
#     else:
#         df = pd.read_csv(filepath + "EURUSD_full_tickstory_data.csv")
#     df_new = pd.read_csv(filepath + df_name)
#     print(len(df_new), df_name)
#     df = pd.concat([df, df_new], ignore_index=True)
#     # save to csv
#     df.to_csv(filepath + "EURUSD_full_tickstory_data.csv", index=False)

In [ ]:
def prep_data_fx(df):
    df = df.reset_index(drop=True)
    # drop any duplicate rows
    df = df.drop_duplicates()
    # convert date to datetime
    df['datetime'] = pd.to_datetime(df['Timestamp'], format='%Y%m%d %H:%M:%S:%f')
    # sort by datetime
    df = df.sort_values(by='datetime')
    # rename bid price to close
    df.rename(columns={'Bid price':'close'}, inplace=True)
    df_ready = df.set_index('datetime')
    # adjust the datetime 7 hrs ahead to match market time
    df_ready.index = df_ready.index + pd.Timedelta(hours=7)
    # ohlc = df_ready[['close']]
    return df_ready

In [ ]:
def generate_renko(df, brick_size):
    # create a renko chart from the ohlc_dec_1 dataframe
    r_full = Renko(df, brick_size=brick_size)
    # create a new dataframe from the renko features
    renko_full_data = r_full.renko_df()
    return renko_full_data

In [ ]:
for csv in csvs:
    if os.path.isfile(f'{filepath}/prepped_{csv}'):
        df = pd.read_csv(f'{filepath}/prepped_{csv}')
        print(f'{csv} already prepped')
    else:
        df = pd.read_csv(filepath + csv)
        df = prep_data_fx(df)
        df.to_csv(f'{filepath}/prepped_{csv}', index=True)
    # find the last day of the dataframe
    last_day = df.index[-1].date()
    # find the last 2 days of the dataframe by adding 1 day to the last day
    last_2_days = last_day - pd.Timedelta(days=1)
    # get the last 2 days of data
    df_last_2_days = df.loc[df.index.date >= last_2_days]
    # save it to a csv file
    df_last_2_days.to_csv(f'{filepath}/prepped_last_2_days_{csv}', index=True)

In [ ]:
# make sure that none of the csvs have overlapping data
for i, csv in enumerate(csvs):
    df = pd.read_csv(f'{filepath}/prepped_last_2_days_{csv}')
    if i == len(csvs) - 1:
        print('we are at the end of the list, stop')
        break
    df2 = pd.read_csv(f'{filepath}/prepped_{csvs[i+1]}')
    if df.index[-1] >= df2.index[0]:
        # drop any rows in df2 that are in df by finding the intersection of the two indexes
        df2 = df2.drop(df2.index.intersection(df.index), axis=0)
    del df
    # save it back to the csv without the overlap
    df2.to_csv(f'{filepath}/prepped_{csvs[i+1]}', index=True)
    del df2
    print(f'finished {i} loop')

In [ ]:
for i, csv in enumerate(csvs):
    df = pd.read_csv(f'{filepath}/prepped_{csv}')
    # convert the 'datetime' column to datetime
    df['datetime'] = pd.to_datetime(df['datetime'])
    # set the datetime column as the index
    df = df.set_index('datetime')
    df_prepped = df[['close']]
    if i == 0:
        renko_full_data = generate_renko(df_prepped, brick_size)
        renko_full_data = renko_full_data.sort_values(by='datetime')
        year = renko_df.index[0].year
        year_end = renko_df.index[-1].year
        # save renko_df to csv
        renko_df.to_csv(f'{filepath}/renko_full_data_{brick_size_str}_{year}_{year_end}.csv')
        # save renko_full_data to csv
        renko_full_data.to_csv(f'{filepath}/renko_full_data_{brick_size_str}.csv', index=True)
    else:
        # find the first value of in the df_prepped that is greater than or equal to the absolute value of the  difference between last_row.close and df_prepped.close
        # first create a new column in df_prepped that is the absolute value of the difference between the close and last_row.close
        df_prepped['diff'] = np.where(abs(df_prepped['close'] - last_row['close'].values[0]) >= brick_size, 1, 0)
        # find the first index where diff is 1
        first_index = df_prepped[df_prepped['diff'] == 1].index[0]
        # get the dataframe from first_index to the end
        df_prepped_adjusted = df_prepped.loc[first_index:]
        # generate the renko chart
        renko_df = generate_renko(df_prepped, brick_size)
        year = renko_df.index[0].year
        year_end = renko_df.index[-1].year
        # save renko_df to csv
        renko_df.to_csv(f'{filepath}/renko_full_data_{brick_size_str}_{year}_{year_end}.csv')
        # concatenate the renko_df to the renko_full_data.csv
        renko_full_data = pd.read_csv(f'{filepath}/renko_full_data_{brick_size_str}.csv')
        # convert the 'datetime' column to datetime
        renko_full_data['datetime'] = pd.to_datetime(renko_full_data['datetime'])
        # reset the index of renko_df
        renko_df = renko_df.reset_index()
        renko_full_data = pd.concat([renko_full_data, renko_df], ignore_index=True)
        # sort by datetime
        renko_full_data = renko_full_data.sort_values(by='datetime')
        # save to csv
        renko_full_data.to_csv(f'{filepath}/renko_full_data_{brick_size_str}.csv', index=False)
    renko_datetime_close = renko_full_data.loc[:, ['datetime', 'close']]
    # get the last row of the renko full data
    last_row = pd.DataFrame(renko_datetime_close.iloc[-1]).T
    del df

In [ ]:
# df = pd.read_csv(filepath + "EURUSD_full_tickstory_data.csv")

In [ ]:
# df19_20 = pd.read_csv(filepath + "EURUSD_11_15_19_to_11_15_20.csv")
# df20_21 = pd.read_csv(filepath + "EURUSD_11_15_20_to_11_15_21.csv")

In [ ]:
# # find out if there are any missing data from df20_21 that is not in df
# missing_data = df20_21.loc[~df20_21['Timestamp'].isin(df['Timestamp'])]
# missing_data

,Timestamp,Bid price,Ask price,Bid volume,Ask volume


In [ ]:
# df = prep_data_fx(df)
# df.to_csv(filepath + "EURUSD_full_tickstory_data_prepped.csv")

In [ ]:
# renko_full_data = generate_renko(df, 0.00081, ending='_initial')

In [ ]:
# renko_full_data = pd.read_csv(f"{filepath}{anytrading_filepath}renko_full_data_81_initial.csv")
# renko_full_data.head()

,datetime,open,high,low,close,volume
0,2017-11-15 09:01:29.782,1.17936,1.18017,1.17846,1.18017,12545.0
1,2017-11-15 09:09:28.118,1.18017,1.18098,1.17990,1.18098,775.0
2,2017-11-15 10:01:52.936,1.18098,1.18179,1.18035,1.18179,4685.0
3,2017-11-15 10:08:27.340,1.18179,1.18260,1.18162,1.18260,793.0
4,2017-11-15 10:50:31.737,1.18260,1.18341,1.18121,1.18341,3910.0
